# Why does count get corrupted while running same code in threads?

Python has a global interpreter lock, which means that only one thread can execute bytecode instructions at an given time. When you switch to a different thred the current execution is frozen.

Consider the two code examples below:

In [ ]:
from threading import Thread

class Counter:
    def __init__(self):
        self.count = 0
        
    def increment(self, by):
        self.count += by
        
def worker(s, how_many, counter):
    for _ in range(how_many):
        counter.increment(1)
        
def run_threads(func, how_many, counter):
    threads = []
    for i in range(5):
        args = (i, how_many, counter)
        thread = Thread(target=func, args=args)
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join
        
how_many = 10 ** 5
counter = Counter()
run_threads(worker, how_many, counter)

In [ ]:
print(counter.count)